# Welcome to Jupyter!

This repo contains an introduction to [Jupyter](https://jupyter.org) and [IPython](https://ipython.org).

Outline of some basics:

* [Notebook Basics](../examples/Notebook/Notebook%20Basics.ipynb)
* [IPython - beyond plain python](../examples/IPython%20Kernel/Beyond%20Plain%20Python.ipynb)
* [Markdown Cells](../examples/Notebook/Working%20With%20Markdown%20Cells.ipynb)
* [Rich Display System](../examples/IPython%20Kernel/Rich%20Output.ipynb)
* [Custom Display logic](../examples/IPython%20Kernel/Custom%20Display%20Logic.ipynb)
* [Running a Secure Public Notebook Server](../examples/Notebook/Running%20the%20Notebook%20Server.ipynb#Securing-the-notebook-server)
* [How Jupyter works](../examples/Notebook/Multiple%20Languages%2C%20Frontends.ipynb) to run code in different languages.

You can also get this tutorial and run it on your laptop:

    git clone https://github.com/ipython/ipython-in-depth

Install IPython and Jupyter:

with [conda](https://www.anaconda.com/download):

    conda install ipython jupyter

with pip:

    # first, always upgrade pip!
    pip install --upgrade pip
    pip install --upgrade ipython jupyter

Start the notebook in the tutorial directory:

    cd ipython-in-depth
    jupyter notebook

In [1]:
pip install bokeh


Note: you may need to restart the kernel to use updated packages.


In [8]:
from ipywidgets import interact
import ipywidgets as widgets
import numpy as np
import requests
from datetime import datetime
import json


import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.ticker import MaxNLocator
#

from bokeh.io import push_notebook, show, output_notebook
from bokeh.layouts import column, gridplot
from bokeh.models import ColumnDataSource, Slider, Button, Toggle, FactorRange
from bokeh.plotting import figure, show
from bokeh.models.widgets import RadioButtonGroup, Select

output_notebook()


Loading BokehJS ...

In [9]:
r = requests.get('https://dhcr.clarin-dariah.eu/api/v1/courses/index?recent')
    
data = r.json()


In [10]:



# Add callback to widgets
def callback(choose, yy): 
    N = yy
    C = 0
    SW = choose
    x_range = plot.x_range
   
    
    #### Disciplines
    dis = set()
    for i in data:
        for j in range(len(i.get('disciplines'))):
            dis.add(i.get('disciplines')[j]['name'])
    dis = list(dis)
    
    dis_dat = []
    for d in dis:
        dis_dat.append( [i for i in data  for j in range(len(i.get('disciplines'))) if d == i.get('disciplines')[j]['name'] ] )            

    dis_dat_year = []
    
    for d in dis_dat:
        dis_dat_year.append(  len([i for i in d  for j in i.get('start_date').split(';')     if (datetime.strptime(j, '%Y-%m-%d') >= datetime.strptime(str(N)+'-09-01', '%Y-%m-%d') and datetime.strptime(j, '%Y-%m-%d') < datetime.strptime(str(N+1)+'-09-01', '%Y-%m-%d') )         ]  )   )
    dis_dat_year = np.array(dis_dat_year)



    #### Objects
    obj = set()
    for i in data:
        for j in range(len(i.get('tadirah_objects'))):
            obj.add(i.get('tadirah_objects')[j]['name'])
    obj = list(obj)
    
    obj_dat = []
    for d in obj:
        obj_dat.append( [i for i in data  for j in range(len(i.get('tadirah_objects'))) if d == i.get('tadirah_objects')[j]['name'] ] )            

    obj_dat_year = []
    
    for d in obj_dat:
        obj_dat_year.append(  len([i for i in d  for j in i.get('start_date').split(';')     if (datetime.strptime(j, '%Y-%m-%d') >= datetime.strptime(str(N)+'-09-01', '%Y-%m-%d') and datetime.strptime(j, '%Y-%m-%d') < datetime.strptime(str(N+1)+'-09-01', '%Y-%m-%d') )         ]  )   )
    obj_dat_year = np.array(obj_dat_year)
    
    #### Techniques
    teq = set()
    for i in data:
        for j in range(len(i.get('tadirah_techniques'))):
            teq.add(i.get('tadirah_techniques')[j]['name'])
    teq = list(teq)
     
    teq_dat = []
    for d in teq:
        teq_dat.append( [i for i in data  for j in range(len(i.get('tadirah_techniques'))) if d == i.get('tadirah_techniques')[j]['name'] ] )            

    teq_dat_year = []
    
    for d in teq_dat:
        teq_dat_year.append(  len([i for i in d  for j in i.get('start_date').split(';')     if (datetime.strptime(j, '%Y-%m-%d') >= datetime.strptime(str(N)+'-09-01', '%Y-%m-%d') and datetime.strptime(j, '%Y-%m-%d') < datetime.strptime(str(N+1)+'-09-01', '%Y-%m-%d') )         ]  )   )
    teq_dat_year = np.array(teq_dat_year)


    if C ==0:
        if SW == "Disciplines":
            x_range.factors = dis
            source.data={'x': dis, 'y': dis_dat_year}
        elif SW == "Objects":
            x_range.factors = obj
            source.data={'x': obj, 'y': obj_dat_year}
        elif SW == "Techniques":
            x_range.factors = teq
            source.data={'x': teq, 'y': teq_dat_year}           
    elif C==1:
        
        if SW == "Disciplines":
            x_range.factors = dis
            if sum(dis_dat_year)==0:
                source.data={'x': dis, 'y': dis_dat_year}
            else:
                source.data={'x': dis, 'y': dis_dat_year/sum(dis_dat_year)}               
        elif SW == "Objects":
            x_range.factors = obj
            if sum(dis_dat_year)==0:
                source.data={'x': obj, 'y': obj_dat_year}
            else:
                source.data={'x': obj, 'y': obj_dat_year/sum(obj_dat_year)}   
        elif SW == "Techniques":
            x_range.factors = teq
            if sum(dis_dat_year)==0:
                source.data={'x': teq, 'y': teq_dat_year}
            else:
                source.data={'x': teq, 'y': teq_dat_year/sum(teq_dat_year)}   

    push_notebook()
    
    
    

#### hier dann eher so: zuerst die daten einmal einlesen...dann eine funktion, die diese daten wieder ausspuckt zusammen mit den labels 
####
#### zuerst muss man die jahre unterteilen...alles was nach 01.SEP und vor 31.AUG liegt gehört zusammen. 
#### damit kann man dann eine years liste machen: years = [1999, 2000, 2001, usw...]
####
#### sturcktur der labels = [  ['hier kommen die course_name labels rein'] , ['hier kommen die diszipline labels rein'] , [usw...]  ]
####
#### strucktur der daten  = [  ['hier kommt die anzahl der jeweiligen labels rein'] , [usw...] , [usw...]  ]
####
#### 






dis = set()
for i in data:
    for j in range(len(i.get('disciplines'))):
        dis.add(i.get('disciplines')[j]['name'])
dis = list(dis)
obj = set()
for i in data:
    for j in range(len(i.get('tadirah_objects'))):
        obj.add(i.get('tadirah_objects')[j]['name'])
obj = list(obj)
teq = set()
for i in data:
    for j in range(len(i.get('tadirah_techniques'))):
        teq.add(i.get('tadirah_techniques')[j]['name'])
teq = list(teq)


####alle start_dates jahre finden...das muss man mal noch genauer machen mit den daten...

min_year = 2100
max_year = 1900

for i in data:
    for j in i.get('start_date').split(';'):
        if datetime.strptime(j, '%Y-%m-%d') >= datetime.strptime(str(max_year+1)+'-09-01', '%Y-%m-%d') :
            max_year = int(j[:4])-1
        if datetime.strptime(j, '%Y-%m-%d') < datetime.strptime(str(min_year)+'-09-01', '%Y-%m-%d') :
            min_year = int(j[:4])-1
            
            
#start the plots with the curretn year...max_year
            
# generate the y data...
### zuerst alle verschiedenen dis:

dis_dat = []
for d in dis:
    dis_dat.append( [i for i in data  for j in range(len(i.get('disciplines'))) if d == i.get('disciplines')[j]['name'] ] )


### start with the max_year


dis_dat_year = []

for d in dis_dat:
    dis_dat_year.append(  len([i for i in d  for j in i.get('start_date').split(';')     if (datetime.strptime(j, '%Y-%m-%d') >= datetime.strptime(str(max_year)+'-09-01', '%Y-%m-%d') and datetime.strptime(j, '%Y-%m-%d') < datetime.strptime(str(max_year+1)+'-09-01', '%Y-%m-%d') )         ]  )   )







source = ColumnDataSource(data={'x': dis, 'y': dis_dat_year})
#
#
#
## Create plots and widgets
plot = figure(title="so in etwa...",x_range=FactorRange(factors=dis),tools="") 
plot.xaxis.major_label_orientation = np.pi/4   # radians, "horizontal", "vertical", "normal"
plot.vbar(x= 'x', top='y', source=source, width=0.5, color = "#ff1200") 
#
#
#
#
#slider = Slider(start=min_year, end=max_year, value=max_year, step=1, title='Academic year starting from winter term')
#slider.on_change('value', callback)
#
#
#rbg = RadioButtonGroup(labels=["Absolute", "Relative"], active=0)
#rbg.on_change('active',callback)
#
#
#select = Select(title="Option:", value="Disciplines", options=["Disciplines", "Objects", "Techniques"])
#select.on_change('value',callback)
#
#    
# 
#
## Arrange plots and widgets in layouts
show(plot,  notebook_handle=True,ncols=1,toolbar_location=None,toolbar_options=dict(logo=None))#, plot_width=250, plot_height=250)


In [11]:
interact(callback, choose=["Disciplines", "Objects", "Techniques"], yy=widgets.IntSlider(min=min_year, max=max_year, step=1, value=max_year))


interactive(children=(Dropdown(description='choose', options=('Disciplines', 'Objects', 'Techniques'), value='…

<function __main__.callback(choose, yy)>